In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output


keras.utils.set_random_seed(253)

Questions for adam:

Is my understanding of the residual correct?
Are there boundary conditions i should be aware of?

check tree paper
assume constant temp
boundary condition: f(t,x), f(t, 0) = 100
initial condition: f(0, x) = f(x) if x > 0

How should I graph this to ensure it's correct?
google solution to equation in 1D

In [ ]:
ALPHA = 1 # thermal diffusivity coefficient
SOURCE_TEMP = 100

t_test = np.arange(0, 5, .1, dtype='float32')
x_test = np.arange(0, 5, .1, dtype='float32')

x_boundary = np.array([0 for item in x_test]) # can technically be used for t_initial, keeping separate for clarity

x_initial = np.arange(.01, 5.01, .01, dtype='float32')
t_initial = np.array([0 for item in x_initial])

list_tensors = []

for item in t_test:
    for item2 in x_test:
        list_tensors.append(np.array([item, item2]))

list_tensors = np.array(list_tensors)


In [ ]:
def make_network() -> tuple:
    layers = []         

    layers.append(keras.layers.Dense(16, activation = 'elu', input_shape = (None, 1, 2)))
    layers.append(keras.layers.Dense(16, activation = 'elu'))
    layers.append(keras.layers.Dense(16, activation = 'elu'))
    layers.append(keras.layers.Dense(16, activation = 'elu'))
    layers.append(keras.layers.Dense(1, activation = 'linear'))
    
    network = keras.Sequential(layers)


    return network


In [ ]:
# residual is how accurate the output is with respect to target equation
def calc_residual(model: keras.Model, xt_list, alpha)-> tf.Tensor:

    with tf.GradientTape(persistent= True) as gt:

        t = xt_list[:, 0:1]
        x = xt_list[:, 1:2]
        t = tf.convert_to_tensor(t)
        x = tf.convert_to_tensor(x)

        t = tf.squeeze(t)
        x = tf.squeeze(x)

        gt.watch(t)
        gt.watch(x)

        u = model(tf.stack([t, x], axis=1))

        du_dx = gt.gradient(u, x)
        # print("u_x: " + str(du_dx))
        # u = tf.convert_to_tensor(tf.squeeze(u), dtype='float32')

    du_dt = gt.gradient(u, t)
    # print("u_t: " + str(du_dt))
    

    d2u_dx2 = gt.gradient(du_dx, x) # second derivative of predicted values
    # print("u_x2: " + str(d2u_dx2))
    del gt

    return du_dt - (alpha * d2u_dx2)


In [ ]:
# calculate how accurate the model is with respect to the actual answers
def calc_loss(model: keras.Model, xt_list, alpha):
    """
    Custom loss function made for neural network. It calculates the residual, and then
    adds it to the difference in the boundary outputs and predicted values.
    Returns the loss.
    
    """

    residual = calc_residual(model, xt_list, alpha)  
    av_residual = tf.reduce_mean(tf.square(residual)) 

    # boundary condition: f(t, x), f(t, 0) = 100 -- heat source of 100 degrees
    # initial condition: f(0, x) = 0 if x > 0 when no heating has occured, rod is 0 degrees when not at heat source

    boundary_output = []
    for item in t_test:
        boundary_output.append(model(tf.stack([[item, 0]])))

    initial_output = []
    for item in x_test:
        initial_output.append(model(tf.stack([[0, item]])))

    boundary_output = tf.convert_to_tensor(boundary_output) # should equal source_temp
    boundary_diff = boundary_output - SOURCE_TEMP

    initial_output = tf.convert_to_tensor(initial_output) # should equal zero

    # print("Av residual: " + str(av_residual))
    # print("Boundary loss: " + str(tf.reduce_mean(tf.square(boundary_diff)) ))
    # print("initial loss: " + str(initial_output))

    
    return av_residual + tf.reduce_mean(tf.square(boundary_diff)) + tf.reduce_mean(tf.square(initial_output))

In [ ]:
# gradient is of the loss function with respect to all the variables in the model
def calc_gradient(model: keras.Model, xt_list, alpha):
    """
    Calculates the gradient to apply to the neural network. Returns the
    gradient.
    
    """

    with tf.GradientTape() as gt:
        gt.watch(model.trainable_variables) # weights
        loss = calc_loss(model, xt_list, alpha)

    gradient = gt.gradient(loss, model.trainable_variables)

    # print(str(gradient))

    return loss, gradient

In [ ]:
def train(model: keras.Model, xt_lists, alpha, optimizer: keras.optimizers.Optimizer):
    """
    Executes one training step for the network. Calculates loss and gradient, applies the 
    gradient, then returns the loss.
    
    """
    
    loss, gradient = calc_gradient(model, xt_lists, alpha)

    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return loss

In [575]:
# train the model
epochs = 1000
i = 0
network = make_network()
# network.load_weights("1D_heat_equation_network.h5")

while i < epochs:

    loss = 0

    loss = train(network, list_tensors, ALPHA, keras.optimizers.Adam())


    print("Epoch: " + str(i))
    print("Loss: " + str(loss))
    print("\n")
    # save every epoch so i can jump ship at any time and still have a network to show for it
    if i % 5 == 0:
        network.save("1D_heat_equation_network.h5")
    i += 1

Epoch: 0
Loss: tf.Tensor(1436.5453, shape=(), dtype=float32)




C:\Users\werea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch: 1
Loss: tf.Tensor(1434.6118, shape=(), dtype=float32)


Epoch: 2
Loss: tf.Tensor(1432.4618, shape=(), dtype=float32)


Epoch: 3
Loss: tf.Tensor(1430.2358, shape=(), dtype=float32)


Epoch: 4
Loss: tf.Tensor(1428.0945, shape=(), dtype=float32)


Epoch: 5
Loss: tf.Tensor(1426.109, shape=(), dtype=float32)


Epoch: 6
Loss: tf.Tensor(1423.9178, shape=(), dtype=float32)


Epoch: 7
Loss: tf.Tensor(1421.778, shape=(), dtype=float32)


Epoch: 8
Loss: tf.Tensor(1419.8457, shape=(), dtype=float32)


Epoch: 9
Loss: tf.Tensor(1417.6758, shape=(), dtype=float32)


Epoch: 10
Loss: tf.Tensor(1415.5181, shape=(), dtype=float32)


Epoch: 11
Loss: tf.Tensor(1413.5894, shape=(), dtype=float32)


Epoch: 12
Loss: tf.Tensor(1411.3494, shape=(), dtype=float32)


Epoch: 13
Loss: tf.Tensor(1409.3207, shape=(), dtype=float32)


Epoch: 14
Loss: tf.Tensor(1407.0625, shape=(), dtype=float32)


Epoch: 15
Loss: tf.Tensor(1405.0946, shape=(), dtype=float32)


Epoch: 16
Loss: tf.Tensor(1403.1298, shape=(), dtyp

In [577]:
network = make_network()
network.load_weights("1D_heat_equation_network.h5")
# y = list(map(lambda a: a*10,y)) # magnification :(


predictions = []

x = tf.convert_to_tensor(x_test, dtype='float32')

for number in range(0, 100):

    t_start = np.array([number for item in x_test])

    t_start = tf.convert_to_tensor(t_start, dtype='float32')
    
    y = network(tf.stack([t_start, x], axis=1))
    predictions.append(y)


plt.ion()

figure = plt.figure()
sub1 = figure.add_subplot(111)
sub1.axis((0, 6, 0, 100))
sub1.set_xlabel("Position")
sub1.set_ylabel("Temp")

line1, = sub1.plot(x, predictions[0], "r--")

for item in predictions:
    line1.set_ydata(item)
    display(figure)
    clear_output(wait=True)
    plt.pause(.2)
    

KeyboardInterrupt: 